## import library
- http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/
- http://learningtensorflow.com/index.html
- http://suriyadeepan.github.io/2016-12-31-practical-seq2seq/

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

# 0. Tensorflow basic

- **tf.constant(), tf.Variable()**: 그래프의 객체
- **tf.Session()/ tf.InteractiveSession()**: 그래프를 시작
- **sess.run(c)**: 텐서 'c'를 계산 -> sess.close() 해야함
- **with 절**: tf.Session()은 with 절과 사용, with 절에서 sess.close() 생략가능
- **c.eval()**: sess.run(c)와 같음

### - **tf.variable_scope()**
- 모델에서 필요한 변수들을 관리하는 클래스(변수 범위 만들기)
- tf.get_variable(): 직접호출 없이 변수를 가져오거나 생성(initializer 사용)
- reuse = False: '현재 variable scope 이름 + 제공된 name'이 없으면 생성
- reuse = True: '현재 variable scope 이름 + 제공된 name'이 있으면 반환
- reuse = AUTO_REUSE: 둘다

# 3. Long Sequence RNN(automatic)

In [212]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

sample = "Hello my name is hyejin"
idx2char = list(set(sample))  # index -> char
# 중복 제거
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex
# character별 숫자 딕셔너리
print(idx2char)
print(char2idx)

['H', ' ', 'h', 'e', 'm', 'y', 'o', 'a', 'i', 'j', 'l', 'n', 's']
{'H': 0, ' ': 1, 'h': 2, 'e': 3, 'm': 4, 'y': 5, 'o': 6, 'a': 7, 'i': 8, 'j': 9, 'l': 10, 'n': 11, 's': 12}


### Set parameters

In [213]:
# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
rnn_hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)

batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1
print(dic_size, rnn_hidden_size, num_classes)
print(sequence_length)

13 13 13
22


### One-hot encoding

In [214]:
sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label
                            #batch_size = 1

# flatten the data (ignore batches for now). No effect if the batch size is 1
x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
print(X)
print(Y)
print(X_one_hot)

Tensor("Placeholder:0", shape=(?, 22), dtype=int32)
Tensor("Placeholder_1:0", shape=(?, 22), dtype=int32)
Tensor("one_hot_6:0", shape=(?, 15, 10), dtype=float32)


C:\Users\user\anaconda3\envs\tf\lib\site-packages\tensorflow_core\python\client\session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


### Create cell

In [215]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

### Train RNN

In [216]:
# reshape out for sequence_loss
print(outputs.shape)
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
print(outputs.shape)

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
prediction = tf.argmax(outputs, axis=2)

(1, 22, 13)
(1, 22, 13)


In [217]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        print(i, "loss:", l, "Prediction:", ''.join(result_str))

0 loss: 2.575422 Prediction: e   l eeeeeeehhhhheenn
1 loss: 2.4515016 Prediction: e     eeeeeeehhhhyhhhh
2 loss: 2.3242366 Prediction: e         mm        nn
3 loss: 2.2703338 Prediction: e  o            nnnnnn
4 loss: 2.182235 Prediction: e ooooooommm  s yynnnn
5 loss: 2.1380637 Prediction: e oo mmmmmmm iiiiynnnn
6 loss: 2.0464647 Prediction: e ommmmmmmmmmmmmiiiiii
7 loss: 2.1022942 Prediction: e    my iiinninninnnin
8 loss: 2.1552591 Prediction: elo  my inne innnnniin
9 loss: 2.0199387 Prediction: ello my nmmm inniyniin
10 loss: 1.9002366 Prediction: ello my nmmm innhyeiin
11 loss: 1.8524112 Prediction: ello my nmmm isnhyennn
12 loss: 1.8225601 Prediction: ello my nmmm ishhyennn
13 loss: 1.7886847 Prediction: ello my nmmm sshhyenin
14 loss: 1.7432549 Prediction: ello my nmmm sshhyeiin
15 loss: 1.7079978 Prediction: ello my nmme ss hyeiin
16 loss: 1.6697178 Prediction: ello my nmme is hyejin
17 loss: 1.6274269 Prediction: ello my nmme is hyejin
18 loss: 1.5950619 Prediction: ello my n